In [1]:
!git clone https://github.com/shoaib6174/MultiWOZ-Clustering/


!pip install pyvis

Cloning into 'MultiWOZ-Clustering'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 113 (delta 56), reused 89 (delta 32), pack-reused 0
Receiving objects: 100% (113/113), 15.81 MiB | 7.58 MiB/s, done.
Resolving deltas: 100% (56/56), done.
Checking out files: 100% (43/43), done.


In [2]:
import pandas as pd
import numpy as np

from pyvis.network import Network
from IPython.core.display import display, HTML

import collections
import math

In [3]:
def show_graph(df, min_weight=1):
    df_edges = df[["cluster_id", "next_utterance_cluster_id"]]
    df_edges = df_edges.dropna()
    df_edges.columns = ['Source', 'Target']
    df_edges_weights = df_edges.groupby(['Source','Target']).size().reset_index().rename(columns={0:'Weight'})
    df_edges_weights = df_edges_weights[df_edges_weights['Weight'] >= min_weight]
    print(len(df_edges_weights.index))
    dialogue_graph = Network(height='750px', width='100%', bgcolor='#222222', font_color='white', directed=True)
    dialogue_graph.barnes_hut()

    sources = df_edges_weights['Source']
    targets = df_edges_weights['Target']
    weights = df_edges_weights['Weight']

    edge_data = zip(sources, targets, weights)



    for edge in edge_data:
        src = edge[0]
        dst = edge[1]
        w = edge[2]

        dialogue_graph.add_node(src, src, title=src)
        dialogue_graph.add_node(dst, dst, title=dst)
        dialogue_graph.add_edge(src, dst, value=w)

    neighbor_map = dialogue_graph.get_adj_list()  #dictionary mapping of Node ID to list of Node IDs 

    # add neighbor data to node hover data (Just provides list of all connected nodes to a given node
    for node in dialogue_graph.nodes:
        node['title'] += ' Neighbors:<br>' + '<br>'.join(neighbor_map[node['id']])
        node['value'] = len(neighbor_map[node['id']])

    dialogue_graph.show('dialogue_graph.html')

    display(HTML("dialogue_graph.html"))

# Import utterances df with clusters and nodes df

In [4]:
path = "/content/MultiWOZ-Clustering/Topic Modeling TF-IDF/dialogue_clusters_TOPIC_200_with_replacement.csv"
df = pd.read_csv(path)
df = df.set_index("utterance_id")
df

,Unnamed: 0,dialogue_id,services,services_list,utterance_no,next_utterance_id,speaker,text,cleaned_text,cluster_id,next_utterance_cluster_id
utterance_id,,,,,,,,,,,
0-SNG02022,0,SNG02022,hospital,['hospital'],0,1-SNG02022,USER,I am looking for the Addenbrookes Hospital wit...,i am looking for the addenbrookes hospital wit...,u-74,s-57
1-SNG02022,1,SNG02022,hospital,['hospital'],1,2-SNG02022,SYSTEM,The telephone number is 01223217712,the telephone number is 01223217712,s-57,u-79
2-SNG02022,2,SNG02022,hospital,['hospital'],2,3-SNG02022,USER,Thank you Goodbye,thank you goodbye,u-79,s-71
3-SNG02022,3,SNG02022,hospital,['hospital'],3,NaN,SYSTEM,Have a wonderful day!,have a wonderful day,s-71,NaN
0-PMUL4452,4,PMUL4452,restaurant & taxi & hotel,"['restaurant', 'taxi', 'hotel']",0,1-PMUL4452,USER,I am traveling to Cambridge and looking forwar...,i am traveling to cambridge and looking forwar...,u-44,s-1
...,...,...,...,...,...,...,...,...,...,...,...
15-MUL2568,33319,MUL2568,taxi & attraction & hotel,"['taxi', 'attraction', 'hotel']",15,16-MUL2568,SYSTEM,I have booked a car from the acorn to country ...,i have booked a car from the acorn to country ...,System No Cluster,u-57
16-MUL2568,33320,MUL2568,taxi & attraction & hotel,"['taxi', 'attraction', 'hotel']",16,17-MUL2568,USER,No that was it. Thanks for your help!,no that was it thanks for your help,u-57,System No Cluster
17-MUL2568,33321,MUL2568,taxi & attraction & hotel,"['taxi', 'attraction', 'hotel']",17,18-MUL2568,SYSTEM,So glad that I could assist. Please call again.,so glad that i could assist please call again,System No Cluster,User No Cluster


In [5]:
pd.set_option("display.max_colwidth", -1)
pd.set_option('display.max_rows', 1000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [6]:
path = "/content/MultiWOZ-Clustering/Topic Modeling TF-IDF/nodes_TOPIC_with_10_utterances.csv"
nodes_df = pd.read_csv(path)
nodes_df.columns = ["cluster_id", "10_utterances"]

nodes_df.head(10)

,cluster_id,10_utterances
0,u-0,"['thank you that s all i needed help with today goodbye', 'just one please', 'yes i need the reference number for the booking please', 'book the one in the centre a table for 2 at 1345 on thursday', 'i d like to leave the hotel by 315 please', 'well what is on the west side then', 'how about asian oriental food', 'ok are there any 4 star hotels with free wifi', 'great can you book that for 7 people for 3 nights starting friday', 'perfect thanks so much for your help today that s everything i needed']"
1,u-1,"['yes i also need to find a train from cambridge to stevenage on friday i need to arrive by 0815', 'are there no international places to dine in the area anything other than british food in the area', 'nope that s it thanks', 'can you provide information about clare hall', 'i am going to cambridge and would like to leave after 1145', 'please book for 4 people at 1700 on sunday', 'i am also looking for a place to go in the centre i would like to see architecture', 'pets on the gray please make the booking for one person and give me the reference number', 'how about for 2 nights', 'can you make a reservation for 6 people for 3 nights starting on wednesday']"
2,u-2,"['i am traveling to cambridge and looking forward to try local restaurants', 'the centre please', 'actually i just need the address of the hotel', 'can you confirm addenbrookes has a medical decisions unit', 'i will do so and you too', 'please help me book a taxi', 'what is the most eastern located 4 star hotel', 'could you provide me with the address and postcode', 'the north please and cheap', 'i prefer the guesthouse please i d like a reservation for 6 people 2 nights we ll be arriving on tuesday']"
3,u-3,"['yes i also need a taxi from the restaurant to the hotel can you help me with that', 'i would like to find a expensive one please', 'can you also find finches bed and breakfast', 'does it have a 4 star rating i would like a place with four stars', 'perfect i ll need 2 tickets', 'that s all i need thank you so much', 'thank you goodbye', 'just to commute between the two places', 'i m looking for a train from stansted airport to cambridge and should arrive by 0930 the same day as the restaurant booking', 'anything in the centre would be fine whatever you recommend']"
4,u-4,"['yeah that would be great thanks', 'can i have the postcode entrance fee and address i also need a hotel within a moderate price range', 'yes i am ready for a reservation', 'i also need information on arbury lodge guesthouse', 'oh yeah five people please', 'yes i d like a taxi to take me to pipasha restaurant and then to cambridge punter i d like to arrive at pipasha restaurant by 1700', 'excellent thanks a lot i think that s everything i need', 'yes the centre is where i will be staying', 'sure can you try just 2 nights please', 'thanks so much can you help me find a train']"
5,u-5,"['i also need a taxi from the restaurant to the hotel please i need to leave the restaurant by 1900', 'i want italian so let s go with pizza hut cherry hinton', 'i d like to leave milton country park by 1015 and go to pizza hut cherry hinton', 'i do nt have a preference', 'yes i would like to stay in a guesthouse that is moderately priced', 'i would also like more info about pizza hut in the city center', 'i need to find an italian restaurant please', 'gosh i m so sorry i do nt want the reservation for the evening i want thai food for lunch do you have a table for 2 at 1230 on tuesday', 'i m not really concerned with price which would you suggest', 'i am looking for a restaurant in the centre of town']"
6,u-6,"['that s alright thanks so much i ve got nothing more today bye', 'yes i ll need a place to sleep off all that food do you have anything that s cheap but also has a 4star rating', 'what is their contact number', 'will i be able to find it with just the street name is that the full address sidney street please provide me with the full address', 'could you recommend me a ho

#Graphs

### all Services

In [31]:
min_edge_weight = 3
show_graph(df, min_edge_weight)

3048


### Service Wise Graph

In [8]:
set(df['services'])

{'attraction',
 'attraction & hotel',
 'attraction & train',
 'hospital',
 'hotel',
 nan,
 'restaurant',
 'restaurant & attraction',
 'restaurant & hotel',
 'restaurant & taxi & attraction',
 'restaurant & taxi & hotel',
 'restaurant & train',
 'taxi',
 'taxi & attraction & hotel',
 'train',
 'train & hotel'}

attraction

In [29]:
# write the name of services from the set above to check graph
services = ['attraction']


df_services = df[df["services"].isin(services)]
df_services.head()

,Unnamed: 0,dialogue_id,services,services_list,utterance_no,next_utterance_id,speaker,text,cleaned_text,cluster_id,next_utterance_cluster_id
utterance_id,,,,,,,,,,,
0-SNG1058,766,SNG1058,attraction,['attraction'],0,1-SNG1058,USER,I'm looking for a place to go in town to have fun. Are there any nightclubs in the west?,i m looking for a place to go in town to have fun are there any nightclubs in the west,u-30,s-38
1-SNG1058,767,SNG1058,attraction,['attraction'],1,2-SNG1058,SYSTEM,"There are no nightclubs in the west, would another area work for you?",there are no nightclubs in the west would another area work for you,s-38,u-18
2-SNG1058,768,SNG1058,attraction,['attraction'],2,3-SNG1058,USER,How about a college in the west?,how about a college in the west,u-18,s-10
3-SNG1058,769,SNG1058,attraction,['attraction'],3,4-SNG1058,SYSTEM,"Sure, there are 5 colleges in the west part of town. How about Clare College?",sure there are 5 colleges in the west part of town how about clare college,s-10,u-57
4-SNG1058,770,SNG1058,attraction,['attraction'],4,5-SNG1058,USER,What's the address of that college?,what s the address of that college,u-57,s-1


In [10]:
min_edge_weight = 1
show_graph(df_services, min_edge_weight)

597


# Repetation per cluster per dialogue

In [11]:
min_number_of_repetation_per_dialogue = 2
rep_per_cluster_per_dialogue = df.groupby(["dialogue_id", 'cluster_id'])["cluster_id"].count().to_frame().add_suffix('_Count').reset_index()
rep_per_cluster_per_dialogue = rep_per_cluster_per_dialogue[rep_per_cluster_per_dialogue["cluster_id_Count"] >= min_number_of_repetation_per_dialogue].reset_index().drop(['index'], axis=1)
rep_per_cluster_per_dialogue.columns = ["dialogue_id", "cluster_id", 'rep_count']
rep_per_cluster_per_dialogue

,dialogue_id,cluster_id,rep_count
0,MUL0005,s-1,2
1,MUL0005,s-54,3
2,MUL0015,s-57,2
3,MUL0015,s-66,2
4,MUL0019,s-1,4
...,...,...,...
3161,WOZ20607,System No Cluster,3
3162,WOZ20607,User No Cluster,4
3163,WOZ20613,s-1,2
3164,WOZ20644,System No Cluster,3


counting rep_count

In [12]:
rep_per_cluster_per_dialogue.drop(['dialogue_id'], axis=1).groupby([ 'rep_count'])['cluster_id'].count().to_frame()

,cluster_id
rep_count,
2,2514
3,398
4,86
5,28
6,42
7,37
8,24
9,13
10,7


# Cluster repetition

### occurrence of cluster repitations sorted

In [13]:
min_Count_of_cluster_id = 1
occurrence_of_cluster_rep_per_cluster = rep_per_cluster_per_dialogue.groupby(['cluster_id'])['cluster_id'].count().to_frame().add_prefix('Count_of_').reset_index().sort_values(['Count_of_cluster_id'],ascending=False )
occurrence_of_cluster_rep_per_cluster.columns = ["cluster_id", 'num_of_repetation_occurances']
occurrence_of_cluster_rep_per_cluster = pd.merge(occurrence_of_cluster_rep_per_cluster, nodes_df, how='left')

occurrence_of_cluster_rep_per_cluster = occurrence_of_cluster_rep_per_cluster[occurrence_of_cluster_rep_per_cluster["num_of_repetation_occurances"] >= min_Count_of_cluster_id]
occurrence_of_cluster_rep_per_cluster.head(10)

,cluster_id,num_of_repetation_occurances,10_utterances
0,s-1,498,"['have a wonderful day', 'i would be happy to help there are lots of great restaurants in town do you what to try a particular type of food', 'there are several restaurants what type of food would you like', 'have a good day', 'i will book it for you now', 'there are 5 trains available leaving at 1559 1759 1959 2159 and 2359 they all take 88 minutes and cost 1660 pounds', 'you are booked your car is a red audi and contact number is 07727005105', 'i m sorry for the mixup your taxi is a white lexus the contact number is 07770889191 is there anything else', 'you can contact them at parkside cambridge cb11jg', 'hotel du vin and bistro serves european food in the moderate price range it is located at 15 19 trumpington street would you like to reserve a table']"
1,u-57,334,"['i m at kettle s yard and need a taxi to pick me up by 1930', 'i am looking for a hospital in town', 'no he taxi should depart from hotel du vin and bistro', 'i d also like to line up a place to eat if you could help me with that', 'yes please i need a table on friday at 1230 for 3 people please', 'sure could i get the address please', 'no that s all i need thank you', 'yeah can you get me a ticket', 'as long as it has free wifi i ll be happy', 'can i get their postcode']"
2,s-57,297,"['the telephone number is 01223217712', 'sure for how many people on what day and time', 'yes the arrival time is 1727', 'you re welcome is there anything else i can help you with today', 'absolutely its an asian oriental restaurant listed as cheap price range located in the centre area would you like me to book a table for you', 'you are welcome to use our services anytime', 'you are set i have made you a reservation at the limehouse for friday night', 'i have booked a table for 3 at 1130 your reference number is our3l525 can i help with anything else', 'all of them are in the centre two are in the cheap price range and two are expensive would any of those suit', 'sure when would you like to leavearrive']"
3,s-66,212,"['i recommend rosa s bed and breakfast could i book a stay for you', 'okay i ve booked for you a yellow skoda the contact number is 07314173329', 'i would be happy to help with your request what is your destination', 'you are more than welcome', 'i m happy to book that for you i just need to know how many nights you ll be staying also how many people', 'thank you for using are service and let us know if you have any future needs', 'there are daily departures from cambridge at 0706 that arrive in petersborough at 0756 which day were you planning to travel', 'what time do you need to depart and on what day', 'alright perfect how many people will need tickets', 'sure the address is grafton hotel 619 newmarket road fen ditton and the post code is cb58pa can i do anything else for you today']"
4,User No Cluster,128,NaN
5,System No Cluster,126,NaN
6,s-54,95,"['there s nothing matching that do you have any other criteria', 'where to and from and what time will you need the taxi', 'we have 19 that match your search now do you have any other preferences', 'gourmet burger kitchen serve north american food do you want me to book a table', 'the post code for gourmet burger kitchen is cb21ab can i help you with anything else today', 'okay what day would you like to travel', 'you are quite welcome have a good day', 'i ll be glad to find you a place first is there an area or price range you recommend', 'well what area are you interested in', 'have a nice day']"
7,s-0,86,"['at what time should it leave', 'booking was successful reference number is 9upu6tfe can i help you with anything else today', 'if you take the tr8224 you ll get to bishop s stortford at 1207', 'their phone number is 01223356354 do you need anything else', 'tr5488 will depart norwich at 1816 on tuesday arriving in cambridge at 1935 would you like to book passage', 'i am sorry it appears that the hotel does not have anything available would you like 

Count of how many times a cluster has repeated in a dialogue

In [32]:
repeated_clusters = rep_per_cluster_per_dialogue.groupby(['cluster_id', "rep_count"])['cluster_id'].count().to_frame().add_suffix('_Count').reset_index()
repeated_clusters.columns = ['cluster_id', 'rep_count_in_single_dialogue', 'count']

repeated_clusters.head(100)

,cluster_id,rep_count_in_single_dialogue,count
0,System No Cluster,2,10
1,System No Cluster,3,24
2,System No Cluster,4,11
3,System No Cluster,5,12
4,System No Cluster,6,21
5,System No Cluster,7,20
6,System No Cluster,8,9
7,System No Cluster,9,6
8,System No Cluster,10,5
9,System No Cluster,11,3


### sorted by  rep_count_in_single_dialogue and count

Read like: {cluster id} has repeated {rep_count_in_single_dialogue} times in {count} dialogues

In [34]:
repeated_clusters_sorted = repeated_clusters.sort_values(['rep_count_in_single_dialogue', 'count'],ascending=False )
repeated_clusters_sorted = pd.merge(repeated_clusters_sorted, nodes_df, how='left')
repeated_clusters_sorted.head(50)

,cluster_id,rep_count_in_single_dialogue,count,10_utterances
0,User No Cluster,16,1,NaN
1,User No Cluster,14,3,NaN
2,System No Cluster,14,2,NaN
3,System No Cluster,13,1,NaN
4,User No Cluster,12,3,NaN
5,System No Cluster,12,2,NaN
6,System No Cluster,11,3,NaN
7,User No Cluster,11,2,NaN
8,System No Cluster,10,5,NaN
9,User No Cluster,10,2,NaN


## dialogues sorted based on highest repetation

Read like: {dialogue_id} has {count} clusters which repeated {repetation_type} times

In [16]:
dialogue_rep = rep_per_cluster_per_dialogue.drop(['cluster_id'], axis=1).groupby([ 'dialogue_id','rep_count' ])['rep_count'].count().to_frame().add_suffix('_Count').reset_index()

dialogue_rep.columns = ["dialogue_id", "repetation_type", "count"]
dialogue_rep = dialogue_rep.sort_values(['repetation_type', 'count'],ascending=False ).reset_index()
dialogue_rep.drop(['index'],axis=1 , inplace=True)

dialogue_rep.head(50)

,dialogue_id,repetation_type,count
0,PMUL0453,16,1
1,MUL0183,14,2
2,MUL1031,14,2
3,PMUL4387,14,1
4,PMUL0453,13,1
5,PMUL2156,12,2
6,MUL0327,12,1
7,PMUL4000,12,1
8,PMUL4387,12,1
9,PMUL2851,11,2


#### find dialogue with ids to check the utterances

In [17]:
dia_ids_to_find = [ "MUL1332" , 'MUL1191' ,'PMUL2587' ]

see repetation count of each cluster for the selected dialogues

In [18]:
rep_count_for_selected_dia = rep_per_cluster_per_dialogue[ rep_per_cluster_per_dialogue["dialogue_id"].isin(dia_ids_to_find)]
rep_count_for_selected_dia

,dialogue_id,cluster_id,rep_count
434,MUL1191,s-0,3
435,MUL1191,s-1,3
436,MUL1191,s-13,2
437,MUL1191,s-42,2
438,MUL1191,s-45,2
439,MUL1191,s-66,2
440,MUL1191,u-2,2
441,MUL1191,u-20,2
442,MUL1191,u-26,3
443,MUL1191,u-35,2


Check top 10 utterances of choosen cluster

In [19]:
cluster_ids = rep_count_for_selected_dia['cluster_id']
#cluster_ids = ["s-21", "s-23", "u-11", "u-41"]

nodes_df[nodes_df['cluster_id'].isin(cluster_ids)]

,cluster_id,10_utterances
2,u-2,"['i am traveling to cambridge and looking forward to try local restaurants', 'the centre please', 'actually i just need the address of the hotel', 'can you confirm addenbrookes has a medical decisions unit', 'i will do so and you too', 'please help me book a taxi', 'what is the most eastern located 4 star hotel', 'could you provide me with the address and postcode', 'the north please and cheap', 'i prefer the guesthouse please i d like a reservation for 6 people 2 nights we ll be arriving on tuesday']"
14,u-14,"['i need to find somewhere to stay in the south', 'i am also looking for a restaurant should be cheap and in the south near to my hotel do you have any options', 'thank you from the hotel to the restaurant please', 'can you help me out with finding a restaurant that serves british food in the east', 'that sounds great can i get the postcode entrance fee and phone number', 'i m also looking for places to go in the east', 'is that located in the north', 'i would like you to book please', 'yes thanks that sounds great i am just interested in the entrance fee', 'ca nt you suggest one and book a room for 2 for 5 nights starting tues please']"
20,u-20,"['what about 3 nights', 'i do nt have one but i d like one of 3 stars', '5 nights starting from thursday for 3 people reference number please', 'i just need one ticket please', 'what is the address and phone number', 'goodbye', 'could you redo booking for 3 people please', 'i am also interested in attractions in the centre specifically i am interested in multiple sports facilities', 'it does nt matter i like 4 star guesthouses though', 'i d prefer something in the moderate price range']"
26,u-26,"['do they have wifi', 'thanks can you also tell me about j restaurant', 'i m looking for lodgings offering free wifi and free parking', 'i want some advice on places to go', 'thank you that was all i needed goodbye', 'no that will be all thank you', 'yes i m headed to cambridge on wednesday', 'no sorry i meant 2 nights', 'the location and price range do nt matter to me i want a guesthouse with 4 stars', 'yes i will need a taxi to go between the two places']"
35,u-35,"['thank you for your help', 'i d like a moderately priced european restaurant in the centre please', 'thank you that was all i needed', 'i would want something expensive that has free parking do either have those', 'i am also looking for a place to go in town college maybe', 'no thanks you were tremendously helpful have a great day', 'i need to arrive by 1215', 'yes i will book that', 'i am looking for a train the train should go to cambridge and should arrive by 1145', 'are there any rooms available in hotels with free parking']"
57,u-57,"['i m at kettle s yard and need a taxi to pick me up by 1930', 'i am looking for a hospital in town', 'no he taxi should depart from hotel du vin and bistro', 'i d also like to line up a place to eat if you could help me with that', 'yes please i need a table on friday at 1230 for 3 people please', 'sure could i get the address please', 'no that s all i need thank you', 'yeah can you get me a ticket', 'as long as it has free wifi i ll be happy', 'can i get their postcode']"
70,u-70,"['you ve given me everything i need thanks goodbye', 'are they in the east or centre', 'i do nt want to book at this time just the information so that s all thanks', 'i am looking for the addenbrookes hospital the hospital should have the paediatric clinic department', 'i was goona ask to search for international but it sounds like their isnt an option with a cheap price lets go with nando s', 'please book reservation for 2 people at 1600 on friday send me the reference number', 'yes i need to find a place to stay with a 4 star rating in the center of town please', 'the centre please', 'starting on wednesday please 2 nights for 2 people', 'what are the prices like there']"
85,s-0,"['at what time should it leave', 'booking was successful reference number is 9upu6tfe can i help 

check the uttereances of selected dialogues

In [20]:
df[df["dialogue_id"].isin(dia_ids_to_find)][["dialogue_id", 'services', "text", "cluster_id", "next_utterance_cluster_id"]]

,dialogue_id,services,text,cluster_id,next_utterance_cluster_id
utterance_id,,,,,
0-MUL1191,MUL1191,attraction & hotel,I am looking for a hotel with free wifi for when I am visiting Cambridge.,u-12,s-5
1-MUL1191,MUL1191,attraction & hotel,Acorn guest house in the north has a 4 star rating and free internet. Would you like me to book a room?,s-5,u-47
2-MUL1191,MUL1191,attraction & hotel,Could you tell me if Acorn guest house is in the moderate price range?,u-47,s-66
3-MUL1191,MUL1191,attraction & hotel,Yes it is in the moderate price range. Shall I book it for you?,s-66,u-20
4-MUL1191,MUL1191,attraction & hotel,"Yes, I need it for 4 people for 3 nights starting Thursday.",u-20,s-18
5-MUL1191,MUL1191,attraction & hotel,"I'm sorry, that is not available. Could you possibly shorten your stay? Also, this is a guesthouse, were you looking for a hotel room instead?",s-18,u-18
6-MUL1191,MUL1191,attraction & hotel,"Yeah, let's just try for one night please.",u-18,s-30
7-MUL1191,MUL1191,attraction & hotel,"Sorry, let me try to get this right. Are you looking for a moderate priced hotel(?) or guesthouse(?), with wifi, in the north? Does star number matter? For 1 night?",s-30,u-57
8-MUL1191,MUL1191,attraction & hotel,I want it to be a hotel and the stars don't matter.,u-57,s-1


# Cluster Flow in dialogue

flow of clusters in dialogues

In [21]:
df_flow = df.groupby(['dialogue_id'], as_index = False).agg({'cluster_id': ' '.join})
df_flow.head(20)

,dialogue_id,cluster_id
0,MUL0002,u-48 s-57 u-30 s-10 u-24 s-71 u-31 s-26 u-40 s-42 u-78 s-38 u-70 s-46
1,MUL0005,u-22 s-30 u-23 s-1 u-53 s-54 u-68 s-66 u-26 s-18 u-29 s-1 u-46 s-54 u-31 s-54 u-54 s-34
2,MUL0007,u-35 s-1 u-78 s-55 u-45 s-42 u-21 s-12 u-10 s-66 u-33 s-36 u-74 s-57
3,MUL0015,u-62 s-66 u-2 s-30 u-56 s-45 u-47 s-53 u-20 s-57 u-31 s-57 u-29 s-54 u-67 s-61 u-30 s-66
4,MUL0019,u-13 s-1 u-66 s-1 u-0 s-1 u-71 s-55 u-18 s-1 u-82 s-12 u-20 s-66 u-26 s-23 u-35 s-5 u-59 s-71
5,MUL0020,u-30 s-31 u-28 s-66 u-20 s-55 u-14 s-23 u-10 s-1 u-65 s-57 u-57 s-1 u-23 s-1 u-71 s-57 u-1 s-34 u-20 s-57
6,MUL0022,u-2 s-1 u-14 s-4 u-78 s-53 u-71 s-34 u-2 s-57 u-77 s-61 u-22 s-50 u-78 s-57 u-57 s-27 u-35 s-38 u-30 s-46
7,MUL0023,u-23 s-54 u-67 s-57 u-23 s-7 u-35 s-57 u-51 s-66 u-67 s-57 u-57 s-66 u-40 s-26 u-70 s-37 u-57 s-0 u-43 s-34 u-70 s-1 u-1 s-1 u-22 s-66 u-77 s-69
8,MUL0024,u-0 s-55 u-67 s-0 u-53 s-34 u-63 s-1 u-65 s-7 u-9 s-1 u-18 s-57 u-68 s-58 u-7 s-46 u-28 s-43 u-57 s-46
9,MUL0030,u-71 s-36 u-18 s-1 u-34 s-1 u-78 s-55


Sorted count of repeating flows

In [22]:
flow_count = df_flow.groupby(['cluster_id'])['dialogue_id'].count().to_frame()
flow_count = flow_count.sort_values(['dialogue_id'], ascending=False).reset_index()

flow_count.columns = ["flow", "count"]
flow_count.head(10)

,flow,count
0,User No Cluster System No Cluster User No Cluster System No Cluster User No Cluster System No Cluster User No Cluster System No Cluster User No Cluster System No Cluster User No Cluster System No Cluster User No Cluster System No Cluster,7
1,User No Cluster System No Cluster User No Cluster System No Cluster User No Cluster System No Cluster,4
2,User No Cluster System No Cluster User No Cluster System No Cluster User No Cluster System No Cluster User No Cluster System No Cluster User No Cluster System No Cluster User No Cluster System No Cluster,3
3,User No Cluster System No Cluster User No Cluster System No Cluster User No Cluster System No Cluster User No Cluster System No Cluster,3
4,User No Cluster System No Cluster User No Cluster System No Cluster User No Cluster System No Cluster User No Cluster System No Cluster User No Cluster System No Cluster,2
5,u-57 s-45 u-53 s-45 u-57 s-15 u-66 s-55 u-2 s-70 u-31 s-53 u-13 s-57 u-77 s-40,1
6,u-57 s-41 u-46 s-12 u-70 s-7 u-78 s-53 u-26 s-43 u-20 s-46 u-14 s-45 u-23 s-66 u-2 s-66 u-57 s-22 u-14 s-57 u-59 s-69 u-11 s-42 u-0 s-45 u-58 s-1 u-64 s-45,1
7,u-57 s-42 u-67 s-0 u-66 s-0 u-2 s-34 u-68 s-45,1
8,u-57 s-42 u-9 s-12 u-57 s-12 u-46 s-24 u-14 s-43 u-77 s-32 u-44 s-61 u-63 s-37 u-71 s-38 u-24 s-46 u-18 s-70 u-22 s-46 u-70 s-66,1
9,u-57 s-43 u-30 s-46 u-14 s-34 u-57 s-57,1


Getting Sub-flows from flows withn at least N turns

In [23]:
def allSubFlowsGreaterThanN(s,n):
    elements = s.split(" ")
    l = len(elements)
    if l < 5:
      return []
    elif l == 5:
      return [s]
    
    #print(elements)
    subs = []
    for i in range(0, l - n-1 ):
      for j in range(i+n, l+1):
        #print(i,j )
        subs.append("|".join(elements[i:j]))
    return subs



In [24]:
min_no_of_turns = 4
flows_substrings = []
for flow in flow_count["flow"]:
  flows_substrings += allSubFlowsGreaterThanN(flow, min_no_of_turns)

In [25]:
len(flows_substrings)

254838

Counting frequency of each sub-flows

In [26]:
frequency = collections.Counter(flows_substrings)
df_freq = pd.DataFrame(dict(frequency).items(), columns = ["sub-flow", "count"])
df_freq['length'] = df_freq.apply(lambda row: math.ceil(len(row['sub-flow'])/5), axis=1)
df_freq

,sub-flow,count,length
0,User|No|Cluster|System,594,5
1,User|No|Cluster|System|No,594,5
2,User|No|Cluster|System|No|Cluster,594,7
3,User|No|Cluster|System|No|Cluster|User,490,8
4,User|No|Cluster|System|No|Cluster|User|No,490,9
...,...,...,...
197154,s-1|u-2|s-5|u-20|s-31|u-79,1,6
197155,s-1|u-2|s-5|u-20|s-31|u-79|s-18,1,7
197156,u-2|s-5|u-20|s-31,1,4
197157,u-2|s-5|u-20|s-31|u-79,1,5


Sorted frequency of subflows

In [27]:
df_freq_sorted = df_freq.sort_values(['count', "length"], ascending=False).reset_index()
df_freq_sorted = df_freq_sorted[df_freq_sorted['count'] > 1]
df_freq_sorted.head(10)

,index,sub-flow,count,length
0,2,User|No|Cluster|System|No|Cluster,594,7
1,0,User|No|Cluster|System,594,5
2,1,User|No|Cluster|System|No,594,5
3,40,No|Cluster|System|No|Cluster,562,6
4,77,Cluster|System|No|Cluster,562,5
5,39,No|Cluster|System|No,562,4
6,116,System|No|Cluster|User|No|Cluster,526,7
7,151,No|Cluster|User|No|Cluster,526,6
8,114,System|No|Cluster|User,526,5
9,115,System|No|Cluster|User|No,526,5


# clusters

In [35]:
cluster_count = df.groupby(['cluster_id'])['text'].count().to_frame().reset_index()
cluster_count.columns = ["cluster_id", "count"]
cluster_count = cluster_count.sort_values(['count'], ascending=False)
cluster_count = pd.merge(cluster_count, nodes_df, how='left')
cluster_count.head(10)

,cluster_id,count,10_utterances
0,s-1,1999,"['have a wonderful day', 'i would be happy to help there are lots of great restaurants in town do you what to try a particular type of food', 'there are several restaurants what type of food would you like', 'have a good day', 'i will book it for you now', 'there are 5 trains available leaving at 1559 1759 1959 2159 and 2359 they all take 88 minutes and cost 1660 pounds', 'you are booked your car is a red audi and contact number is 07727005105', 'i m sorry for the mixup your taxi is a white lexus the contact number is 07770889191 is there anything else', 'you can contact them at parkside cambridge cb11jg', 'hotel du vin and bistro serves european food in the moderate price range it is located at 15 19 trumpington street would you like to reserve a table']"
1,u-57,1526,"['i m at kettle s yard and need a taxi to pick me up by 1930', 'i am looking for a hospital in town', 'no he taxi should depart from hotel du vin and bistro', 'i d also like to line up a place to eat if you could help me with that', 'yes please i need a table on friday at 1230 for 3 people please', 'sure could i get the address please', 'no that s all i need thank you', 'yeah can you get me a ticket', 'as long as it has free wifi i ll be happy', 'can i get their postcode']"
2,s-57,1384,"['the telephone number is 01223217712', 'sure for how many people on what day and time', 'yes the arrival time is 1727', 'you re welcome is there anything else i can help you with today', 'absolutely its an asian oriental restaurant listed as cheap price range located in the centre area would you like me to book a table for you', 'you are welcome to use our services anytime', 'you are set i have made you a reservation at the limehouse for friday night', 'i have booked a table for 3 at 1130 your reference number is our3l525 can i help with anything else', 'all of them are in the centre two are in the cheap price range and two are expensive would any of those suit', 'sure when would you like to leavearrive']"
3,s-66,1207,"['i recommend rosa s bed and breakfast could i book a stay for you', 'okay i ve booked for you a yellow skoda the contact number is 07314173329', 'i would be happy to help with your request what is your destination', 'you are more than welcome', 'i m happy to book that for you i just need to know how many nights you ll be staying also how many people', 'thank you for using are service and let us know if you have any future needs', 'there are daily departures from cambridge at 0706 that arrive in petersborough at 0756 which day were you planning to travel', 'what time do you need to depart and on what day', 'alright perfect how many people will need tickets', 'sure the address is grafton hotel 619 newmarket road fen ditton and the post code is cb58pa can i do anything else for you today']"
4,User No Cluster,757,NaN
5,s-54,743,"['there s nothing matching that do you have any other criteria', 'where to and from and what time will you need the taxi', 'we have 19 that match your search now do you have any other preferences', 'gourmet burger kitchen serve north american food do you want me to book a table', 'the post code for gourmet burger kitchen is cb21ab can i help you with anything else today', 'okay what day would you like to travel', 'you are quite welcome have a good day', 'i ll be glad to find you a place first is there an area or price range you recommend', 'well what area are you interested in', 'have a nice day']"
6,System No Cluster,740,NaN
7,u-70,737,"['you ve given me everything i need thanks goodbye', 'are they in the east or centre', 'i do nt want to book at this time just the information so that s all thanks', 'i am looking for the addenbrookes hospital the hospital should have the paediatric clinic department', 'i was goona ask to search for international but it sounds like their isnt an option with a cheap price lets go with nando s', 'please book reservation for 2 people at 1600 on friday send me the refer